In [2]:
import numpy as np
import itertools

In [3]:
# |0> = [1,0], |1> = [0,1]
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

In [28]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on a tensor products of qubit states.

    Intputs:
    f - tensor product of qubit states, in form [[qubit 1 state], [qubit 2 state], ...]
    """

    N = f.shape[0]
    fInv = np.empty([N,2],dtype=np.complex_)
    fInv[-1] = H @ f[-1]

    for i in reversed(range(N-1)): 
        s = f[i]
        for k in range(i+1,N):
            if np.allclose(fInv[k],np.array([0,1])):
                s[1] *= np.exp(-2*np.pi*complex(0,1)/(2**(k+1)))
        s = H @ s
        fInv[i] = s
    
    return fInv


In [29]:
def findNumber(s): #rename to convert from binary
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    binaryForm = np.empty(s.shape[0])
    for i,ket in enumerate(s):
        if np.allclose(ket,np.array([1,0])):
            binaryForm[i] = 0
        elif np.allclose(ket,np.array([0,1])):
            binaryForm[i] = 1
        else:
            print("error")
    
    number = sum([b*2**i for i,b in enumerate(np.flip(binaryForm))])
    return number

        

In [30]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
invf = np.array([[1,complex(0,1)],[1,-1],[1,1]])/np.sqrt(2) 
print(findNumber(invQFT(invf)))

2.0


In [32]:
f = [np.array([1,complex(0,1)]),np.array([1,-1])]/np.sqrt(2)
print(findNumber(invQFT(f))) #should be 1

1.0


In [34]:

def phaseEstimation(t,U,e): 
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    #cache?
    def pow(n,prev=np.identity(N)):
        next = U @ prev
        if n == 1:
            return next
        return pow(n-1,next)
    

    f = np.zeros([t,2])

    for i in range(t):
        s = np.array([1,0])
        s = H @ s

        #apply 
        s[1] *= e @ pow(2**i) @ e
        
        f[t-1-i] = s

    return findNumber(invQFT(f)) #apply inverse QFT


In [35]:
#test for phase esimation, identity matrix, result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0])
phaseEstimation(2,U1,e1) #need to look at this bit

0.0

In [36]:
U2 = np.array([[complex(0,1),0],[0,1]])
e2 = np.array([1,0])
phaseEstimation(2,U2,e2)

error


C:\Users\willi\AppData\Local\Temp\ipykernel_15408\3901862146.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  s[1] *= e @ pow(2**i) @ e


3.0

In [102]:
def shor(N,qNum,phaseAttempts,orderAttempts,psiAcc):
    HDim = 2**qNum

    for i in range(orderAttempts):
        print(i)
        a = np.random.randint(2,high=N)
        K = np.gcd(a,N)
        if K != 1:
            return N/K,K
        
        #find U_a matrix
        U_a = np.zeros([HDim,HDim],dtype=np.complex_)
        for j,k in itertools.product(range(HDim), repeat=2):
            if j == a*k % N:
                U_a[j,k] = 1
        
        #random number generator models quantum probablities
        eigenVal, eigenVec = np.linalg.eig(U_a)
        rs = []
        print("got")
        for l in range(phaseAttempts):
            index = np.random.randint(0,high=HDim)
            print(eigenVal[index])
            phaseEst = getNumber(phaseEstimation(psiAcc,U_a,eigenVec[index]))
            print(phaseEst)
            if phaseEst != None:
                rs.append(index/phaseEst)
        if rs == []:
            continue

        R = np.lcm.reduce(rs)
        print("got here")

        if R % 2 == 0:
            g = np.gcd(a**(R/2) + 1,N)
            if g != 1:
                return N/g, g
            
    return "failed to factorize"


IndentationError: expected an indented block (1175582580.py, line 3)